In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ---------------------------
# 1. Load data
# ---------------------------
df = pd.read_excel("Brent_Price.xlsx", header=1)

# ---------------------------
# 2. Clean 'Date' column
# ---------------------------
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date', 'Close'])

# ---------------------------
# 3. Sort and add columns
# ---------------------------
df = df.sort_values('Date')
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
df['DayOfYear'] = df['Date'].dt.dayofyear

# ---------------------------
# 4. Rebase function
# ---------------------------
def rebase(series):
    return (series / series.iloc[0]) * 100

df['Rebased'] = df.groupby(['Year', 'Quarter'])['Close'].transform(rebase)

# ---------------------------
# 5. Choose years to highlight (up to 3)
# ---------------------------
highlight_years = [2025, 2024, 2023]  # <-- change or reorder as needed
highlight_colors = ['crimson', 'royalblue', 'darkorange']  # colors for each highlighted year
muted_color = 'lightgray'

# ---------------------------
# 6. Create 4 subplots
# ---------------------------
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Q1", "Q2", "Q3", "Q4"),
    shared_yaxes=True,
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

subplot_positions = {1: (1, 1), 2: (1, 2), 3: (2, 1), 4: (2, 2)}

for quarter in range(1, 5):
    quarter_data = df[df['Quarter'] == quarter]
    row, col = subplot_positions[quarter]

    for year, group in quarter_data.groupby('Year'):
        if year in highlight_years:
            idx = highlight_years.index(year)
            color = highlight_colors[idx]
            line_width = 3
        else:
            color = muted_color
            line_width = 1.5

        fig.add_trace(
            go.Scatter(
                x=group['DayOfYear'],
                y=group['Rebased'],
                mode='lines',
                name=str(year),
                line=dict(color=color, width=line_width),
                showlegend=True if quarter == 1 else False
            ),
            row=row, col=col
        )

fig.update_layout(
    height=700, width=1000,
    title_text=f"Brent Price - Yearly Rebased by Quarter (Highlighted: {', '.join(map(str, highlight_years))})",
    template="plotly_white"
)

# Shared Y-axis label
fig.update_yaxes(title_text="Rebased Price (first day = 100)", row=1, col=1)
fig.update_xaxes(title_text="Day of Year", row=2, col=1)
fig.update_xaxes(title_text="Day of Year", row=2, col=2)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# ---------------------------
# 1. Load data
# ---------------------------
df = pd.read_excel("Brent_Price.xlsx", header=1)

# ---------------------------
# 2. Clean 'Date' column
# ---------------------------
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date', 'Close'])

# ---------------------------
# 3. Sort and add columns
# ---------------------------
df = df.sort_values('Date')
df['Year'] = df['Date'].dt.year
df['DayOfYear'] = df['Date'].dt.dayofyear
df['Quarter'] = df['Date'].dt.quarter

# ---------------------------
# 4. Rebase function (per year)
# ---------------------------
def rebase(series):
    return (series / series.iloc[0]) * 100

df['Rebased'] = df.groupby('Year')['Close'].transform(rebase)

# ---------------------------
# 5. Choose years to highlight
# ---------------------------
highlight_years = [2025, 2024, 2023]
highlight_colors = ['crimson', 'royalblue', 'darkorange']
muted_color = 'lightgray'

# ---------------------------
# 6. Create yearly plot
# ---------------------------
fig = go.Figure()

# Add quarterly shaded regions
quarter_colors = ['#e0f3f8', '#ccebc5', '#fddbc7', '#f2f0f7']  # semi-transparent colors
quarter_ranges = {
    1: (1, 90),
    2: (91, 181),
    3: (182, 273),
    4: (274, 366)
}

for q, (start, end) in quarter_ranges.items():
    fig.add_vrect(
        x0=start, x1=end,
        fillcolor=quarter_colors[q-1],
        opacity=0.2,
        layer="below",
        line_width=0,
        annotation_text=f"Q{q}", annotation_position="top"
    )

# Plot yearly rebased lines
for year, group in df.groupby('Year'):
    if year in highlight_years:
        idx = highlight_years.index(year)
        color = highlight_colors[idx]
        line_width = 3
    else:
        color = muted_color
        line_width = 1.5

    fig.add_trace(
        go.Scatter(
            x=group['DayOfYear'],
            y=group['Rebased'],
            mode='lines',
            name=str(year),
            line=dict(color=color, width=line_width)
        )
    )

fig.update_layout(
    height=600, width=900,
    title_text=f"Brent Price - Yearly Rebased with Quarterly Highlights",
    xaxis_title="Day of Year",
    yaxis_title="Rebased Price (first day = 100)",
    template="plotly_white"
)

fig.show()


: 